In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import holidays

In [3]:
count=[[0 for _ in range(44)]for i in range(144)]
x=[]
country_code={
    "Australia":"AUS",
    "Bangladesh":"BD",
    "Canada":"CAN",
    "Germany":"DEU",
    "India":"IND",
    "Japan":"JPN",
    "Switzerland":"CHE",
    "UnitedStates":"USA"
}

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
def prepare_data(path):
    data=pd.read_csv(path)
    x_part=data[['Timestamp','ProtocolName']]
    x.extend(np.array(x_part))

In [6]:
prepare_data('/home/chinmay/Desktop/Samsung/sample_data1.csv')
prepare_data('/home/chinmay/Desktop/Samsung/sample_data2.csv')
prepare_data('/home/chinmay/Desktop/Samsung/sample_data3.csv')
prepare_data('/home/chinmay/Desktop/Samsung/sample_data4.csv')
prepare_data('/home/chinmay/Desktop/Samsung/sample_data5.csv')
prepare_data('/home/chinmay/Desktop/Samsung/sample_data6.csv')
prepare_data('/home/chinmay/Desktop/Samsung/sample_data7.csv')
prepare_data('/home/chinmay/Desktop/Samsung/sample_data8.csv')
prepare_data('/home/chinmay/Desktop/Samsung/sample_data9.csv')

In [7]:
x=np.array(x)
np.random.shuffle(x)
LE=LabelEncoder()
x[:,1]=LE.fit_transform(x[:,1])
for i in range(len(x)):
    x[i][0]=(max((datetime.strptime(x[i][0],'%d/%m/%Y%H:%M:%S').hour-1)*6,0))+((datetime.strptime(x[i][0],'%d/%m/%Y%H:%M:%S').minute)//10)
    count[x[i][0]][x[i][1]]+=1

In [8]:
protocols=list(LE.classes_)
print(protocols)
print(len(protocols))

['AMAZON', 'APPLE', 'APPLE_ICLOUD', 'APPLE_ITUNES', 'CITRIX', 'CLOUDFLARE', 'CONTENT_FLASH', 'DNS', 'DROPBOX', 'EASYTAXI', 'EBAY', 'FACEBOOK', 'FTP_DATA', 'GMAIL', 'GOOGLE', 'GOOGLE_MAPS', 'HTTP', 'HTTP_CONNECT', 'HTTP_DOWNLOAD', 'HTTP_PROXY', 'INSTAGRAM', 'IP_ICMP', 'MAIL_IMAPS', 'MICROSOFT', 'MQTT', 'MSN', 'MSSQL', 'MS_ONE_DRIVE', 'NTP', 'OFFICE_365', 'OPENSIGNAL', 'SKYPE', 'SPOTIFY', 'SSL', 'SSL_NO_CERT', 'TEAMVIEWER', 'TOR', 'TWITTER', 'UBUNTUONE', 'WHATSAPP', 'WIKIPEDIA', 'WINDOWS_UPDATE', 'YAHOO', 'YOUTUBE']
44


In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler2=MinMaxScaler(feature_range=(1,44))

In [10]:
k=0
def prioritize(path):
    global k
    data=pd.read_csv(path)
    data["weightage"]='4'
    for i in range(k,len(data)):
        weight=0.7*(max((datetime.strptime(data['Timestamp'][i],'%d/%m/%Y%H:%M:%S').hour-1)*6,0)+(datetime.strptime(data['Timestamp'][i],'%d/%m/%Y%H:%M:%S').minute//10)*protocols.index(data['ProtocolName'][i]))
        if not datetime.date(datetime.strptime(data['Timestamp'][i],'%d/%m/%Y%H:%M:%S')) in holidays.CountryHoliday(country_code[data['Country'][i]]):
            weight=weight*3/7
        data["weightage"][i]=0.3*count[x[k+i][0]][x[k+i][1]]+weight
    data["weightage"]=np.around(scaler2.fit_transform(np.array(data["weightage"]).reshape(-1,1)))
    data.to_csv(path,index=False)
    k+=len(data)

In [11]:
prioritize('/home/chinmay/Desktop/Samsung/sample_data1.csv')
prioritize('/home/chinmay/Desktop/Samsung/sample_data2.csv')
prioritize('/home/chinmay/Desktop/Samsung/sample_data3.csv')
prioritize('/home/chinmay/Desktop/Samsung/sample_data4.csv')
prioritize('/home/chinmay/Desktop/Samsung/sample_data5.csv')
prioritize('/home/chinmay/Desktop/Samsung/sample_data6.csv')
prioritize('/home/chinmay/Desktop/Samsung/sample_data7.csv')
prioritize('/home/chinmay/Desktop/Samsung/sample_data8.csv')
prioritize('/home/chinmay/Desktop/Samsung/sample_data9.csv')

/home/chinmay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/chinmay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [12]:
j,priority=[],[[0 for _ in range(44)]for i in range(144)]
for i in count:
    j.append((-np.array(i)).argsort())
for d in range(len(j)):
    k=1
    for m in range(len(j[0])):
        priority[d][j[d][m]]=k
        k+=1   
y=[]
for i in range(len(x)):
    y.append(priority[x[i][0]][x[i][1]])

In [13]:
print(len(x),len(x[0]))

180006 2


In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [15]:
print(len(x_train),len(x_train[0]),len(x_test),len(x_test[0]),len(y_train),len(y_test))

144004 2 36002 2 144004 36002


In [16]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier

model1=LogisticRegression()
model2=DecisionTreeClassifier(max_depth=20)
model3=RandomForestClassifier(n_jobs=-1,n_estimators=100,max_depth=20)
model4=Perceptron()
model5=KNeighborsClassifier(n_neighbors=37)

In [18]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)
model4.fit(x_train,y_train)
model5.fit(x_train,y_train)

/home/chinmay/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=37, p=2,
                     weights='uniform')

In [19]:
y_pred1=model1.predict(x_test)
y_pred2=model2.predict(x_test)
y_pred3=model3.predict(x_test)
y_pred4=model4.predict(x_test)
y_pred5=model5.predict(x_test)

In [20]:
print("Multilayer Perceptron Accuracy : ",100*len([x for x in y_test==y_pred4 if x==True])/len(y_test))
print("Logstic Regression Accuracy : ",100*len([x for x in y_test==y_pred1 if x==True])/len(y_test))
print("Decision Tree Accuracy : ",100*len([x for x in y_test==y_pred2 if x==True])/len(y_test))
print("Random Forest Accuracy : ",100*len([x for x in y_test==y_pred3 if x==True])/len(y_test))
print("K Nearest Neighbours Accuracy : ",100*len([x for x in y_test==y_pred5 if x==True])/len(y_test))

Multilayer Perceptron Accuracy :  44.825287484028664
Logstic Regression Accuracy :  54.63863118715627
Decision Tree Accuracy :  99.99722237653464
Random Forest Accuracy :  99.99722237653464
K Nearest Neighbours Accuracy :  99.76945725237486
